In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem, Draw
from rdkit.Chem.Draw import SimilarityMaps
from torch_geometric.data import Data
from IPython.display import Image, display

In [ ]:
def morgan_bit_to_atom_map(mol, radius=2, nBits=2048):
    bitInfo = {}
    _ = AllChem.GetMorganFingerprintAsBitVect(
        mol,
        radius=radius,
        nBits=nBits,
        bitInfo=bitInfo
    )

    bit_atom_map = {}

    for bit, info in bitInfo.items():
        atoms = set()
        for atom_idx, rad in info:
            atoms.add(atom_idx)
        bit_atom_map[bit] = atoms

    return bit_atom_map

In [ ]:
def compute_bit_importance(model, fps_input):
    model.eval()

    fps_input_tensor = fps_input.clone().detach().to(DEVICE)
   
    all_importances_per_mol = []

    
    if not graphs: 
        raise ValueError("Global 'graphs' variable is empty or not defined.")
       
    sample_graph_template = graphs[0].to(DEVICE).clone()
    
    sample_graph_template.x.requires_grad_(False)
    sample_graph_template.edge_index.requires_grad_(False)
    if sample_graph_template.edge_attr is not None:
        sample_graph_template.edge_attr.requires_grad_(False)
   
    for i in range(fps_input_tensor.shape[0]):
        current_fp_single_mol = fps_input_tensor[i:i+1].clone()
        current_fp_single_mol.requires_grad_(True) 

        dummy_data_for_mol = Data(
            x=sample_graph_template.x,
            edge_index=sample_graph_template.edge_index,
            edge_attr=sample_graph_template.edge_attr,
            fp=current_fp_single_mol 
        )
        dummy_data_for_mol.batch = torch.zeros(sample_graph_template.x.size(0), dtype=torch.long, device=DEVICE)
        dummy_data_for_mol.num_graphs = 1

        _, _, logits = model(dummy_data_for_mol) 
       
        output_scalar = logits.mean() 

        model.zero_grad()
       
        output_scalar.backward()

        if current_fp_single_mol.grad is not None:
            importance = (current_fp_single_mol.grad * current_fp_single_mol).detach().cpu().numpy()
            all_importances_per_mol.append(importance.flatten())
        else:
            all_importances_per_mol.append(np.zeros(fps_input_tensor.shape[1]))
           
    if all_importances_per_mol:
        return np.array(all_importances_per_mol).mean(axis=0)
    else:
        return np.zeros(fps_input_tensor.shape[1])

In [ ]:
bit_importance = compute_bit_importance(model, fps)
print("Bit importance shape:", bit_importance.shape)

In [ ]:
def compute_atom_importance(smiles, bit_importance, radius=2, nBits=2048):
    mol = Chem.MolFromSmiles(smiles)
    bit_atom_map = morgan_bit_to_atom_map(mol, radius, nBits)

    atom_scores = np.zeros(mol.GetNumAtoms())

    for bit, atoms in bit_atom_map.items():
        for atom in atoms:
            atom_scores[atom] += bit_importance[bit]

    if np.max(np.abs(atom_scores)) > 0:
        atom_scores /= np.max(np.abs(atom_scores))

    return atom_scores

In [ ]:
def visualize_atom_importance(smiles, atom_scores):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        raise ValueError("Invalid SMILES string")


    atom_scores = atom_scores.tolist()

    drawer = Draw.MolDraw2DCairo(300, 250) 

    SimilarityMaps.GetSimilarityMapFromWeights(
        mol,
        atom_scores,
        colorMap=plt.cm.coolwarm,
        contourLines=10,
        draw2d=drawer 
    )

    drawer.FinishDrawing()
    png = drawer.GetDrawingText()
    
    display(Image(png))

In [ ]:
smiles_example = smiles[0] 

atom_scores = compute_atom_importance(
    smiles_example,
    bit_importance
)

visualize_atom_importance(smiles_example, atom_scores)

In [ ]:
from collections import defaultdict
from rdkit.Chem import AllChem

def get_morgan_bits(smile, radius=2, nBits=2048):
    mol = Chem.MolFromSmiles(smile)
    bitInfo = {}
    AllChem.GetMorganFingerprintAsBitVect(
        mol, radius, nBits=nBits, bitInfo=bitInfo
    )
    return set(bitInfo.keys()), bitInfo


In [ ]:
active_bits = defaultdict(int)
inactive_bits = defaultdict(int)

for s, y in zip(smiles, labels.numpy()):
    bits, _ = get_morgan_bits(s)
    for b in bits:
        if y == 1:
            active_bits[b] += 1
        else:
            inactive_bits[b] += 1


In [ ]:
from scipy.stats import fisher_exact

results = []

N_active = int((labels == 1).sum())
N_inactive = int((labels == 0).sum())

for bit in active_bits:
    a = active_bits[bit]
    b = inactive_bits.get(bit, 0)
    c = N_active - a
    d = N_inactive - b

    _, p = fisher_exact([[a, b], [c, d]], alternative='greater')

    results.append((bit, a, b, p))


In [ ]:
import pandas as pd
from statsmodels.stats.multitest import multipletests

df = pd.DataFrame(results, columns=["bit", "active", "inactive", "p"])
df["q"] = multipletests(df["p"], method="fdr_bh")[1]

enriched = df[df["p"] < 0.01].sort_values("p")

enriched.head(10)


In [ ]:
def draw_bit_substructure(smile, bit):
    mol = Chem.MolFromSmiles(smile)
    _, bitInfo = get_morgan_bits(smile)
    if bit not in bitInfo:
        return None

    atom_ids = [a for a, r in bitInfo[bit]]
    return Draw.MolToImage(
        Chem.PathToSubmol(mol, atom_ids), size=(200, 200)
    )


In [ ]:
print("Number of enriched substructures:", len(enriched))
enriched.head(10)


In [ ]:
top_bit = enriched.iloc[0]["bit"]

for s, y in zip(smiles, labels.numpy()):
    if y == 1:
        bits, bitInfo = get_morgan_bits(s)
        if top_bit in bits:
            img = draw_bit_substructure(s, top_bit)
            display(img)
            break
